problem 2: Finding the Most Attacked Region

In [1]:
import sklearn
from sklearn.preprocessing import StandardScaler
print(sklearn.__version__)  # This will print the version of scikit-learn installed


1.6.1
